In [ ]:
from netgen.occ import *
import netgen.meshing as meshing
from ngsolve import *
from ngsolve.webgui import Draw
from libbem import *
from ngsolve import Projector, Preconditioner
from ngsolve.krylovspace import CG

In [ ]:
sp = Sphere( (0,0,0), 1)
mesh = Mesh( OCCGeometry(sp).GenerateMesh(maxh=1, perfstepsend=meshing.MeshingStep.MESHSURFACE)).Curve(2)

In [ ]:
fesHCurl = HCurl(mesh, order=1, complex=True)
uHCurl,vHCurl = fesHCurl.TnT() # H(curl_Gamma) conform spaces
print ("ndof HCurl = ", fesHCurl.ndof)

In [ ]:
eps0 = 8.854e-12 
mu0 = 4*pi*1e-7
omega = 1.5e9
kappa = omega*sqrt(eps0*mu0)
print("kappa = ", kappa)

E = CF((1,0,0))*exp( -1j * kappa * z )
n = specialcf.normal(3)
Edir = Cross(n,E) # nxE in H(div_Gamma)
Draw (Edir.real, mesh, draw_vol=False, order=2) 
Draw (Edir.imag, mesh, draw_vol=False, order=2)

In [ ]:
rhs = LinearForm( Cross(Edir,n) * Cross(n,vHCurl.Trace()) *ds(bonus_intorder=3)).Assemble() # <H(curl_Gamma), H(div_Gamma)> 

In [ ]:
j = GridFunction(fesHCurl) # note: we actually solve for coefficients for FE space H(div_Gamma)
pre = BilinearForm(uHCurl.Trace()*vHCurl.Trace()*ds).Assemble().mat.Inverse(freedofs=fesHCurl.FreeDofs()) # to be changed
with TaskManager(): # pajetrace=1000*1000*1000):
    SVec = MaxwellVecSingleLayerPotentialOperator(fesHCurl, kappa, 
                                               intorder=12, leafsize=40, eta=3., eps=1e-11, method="svd", testhmatrix=False)
    SSca = MaxwellScaSingleLayerPotentialOperator(fesHCurl, kappa, 
                                               intorder=12, leafsize=40, eta=3., eps=1e-11, method="svd", testhmatrix=False)
    CG(mat = SVec.mat - 1/(kappa*kappa)*SSca.mat, pre=pre, rhs = rhs.vec, sol=j.vec, 
       tol=1e-8, maxsteps=500, initialize=False, printrates=True)

In [ ]:
Draw (j.real, mesh, draw_vol=False, order=2)
Draw (j.imag, mesh, draw_vol=False, order=2)